In [ ]:
import numpy as np 
import pandas as pd
import os # for directories
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
#from tensorflow.keras import 
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D






In [ ]:
#print(os.getcwd())

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
%ls /gdrive/MyDrive/5644_ML_Project/


dataset_dogs_vs_cats/  Dataset_SkinCancer/  model.hdf5  Models_1/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# to get data
train_path = os.path.join('/gdrive/MyDrive/5644_ML_Project', 'Dataset_SkinCancer/train')
print(train_path)
test_path = os.path.join('/gdrive/MyDrive/5644_ML_Project', 'Dataset_SkinCancer/test')
print(test_path)

/gdrive/MyDrive/5644_ML_Project/Dataset_SkinCancer/train
/gdrive/MyDrive/5644_ML_Project/Dataset_SkinCancer/test


In [ ]:
class_names = ['malignant', 'benign']
print("Classes",class_names)

print('Count of Train Images:')
for i in class_names:
    print(i +' Train'+ ':' + str(len(os.listdir(os.path.join(train_path, i)))))

    
print('Count of Test Images:')
for i in class_names:
    print(i +' Test'+ ':' + str(len(os.listdir(os.path.join(test_path, i)))))
    

Classes ['malignant', 'benign']
Count of Train Images:
malignant Train:1197
benign Train:1440
Count of Test Images:
malignant Test:300
benign Test:360


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.2) #,horizontal_flip=True)#)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(32,32),
        batch_size=16,
        class_mode='binary',
        subset='training')
validation_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(32,32),
        batch_size=10,
        class_mode='binary',
        subset='validation')
test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(32,32),
        batch_size=8,
        class_mode='binary')

Found 2110 images belonging to 2 classes.
Found 527 images belonging to 2 classes.
Found 660 images belonging to 2 classes.


In [ ]:
# CNN model

#def model_1 (Malignant_train_images, Malignant_test_images):
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation = "relu", padding='same', input_shape = (32,32,3), trainable=False),
    tf.keras.layers.MaxPooling2D(2,2),
    #tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation = "relu",  trainable=True),
    #tf.keras.layers.MaxPooling2D(2,2),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv2D(128, (3,3), padding='same', activation = "relu", trainable=True),
    #tf.keras.layers.MaxPooling2D(2,2),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256,activation = "relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(80,activation = "relu"),
    tf.keras.layers.Dense(1,activation = "sigmoid")
])


# load json and create model
# json_file = open('model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
#loaded_model = model_from_json(loaded_model_json)

# load weights into new model
model.load_weights(r"/gdrive/MyDrive/5644_ML_Project/Models_1/cat_dogs_model_v1.h5")
print("Loaded model from disk")


    # training the model    
opt = keras.optimizers.Adam(learning_rate=0.00001)
model.summary()

model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['acc'])

# checkpoint
#filepath="best_weights_p_neuron-{epoch:02d}-{val_acc:.2f}.hdf5"
filepath = r"/gdrive/MyDrive/5644_ML_Project/Models_1/transfter_model_v2"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')
early_stop = EarlyStopping (monitor='val_acc', mode='max', verbose=0, patience=10)
callbacks_list = [early_stop,checkpoint]

history = model.fit_generator(train_generator, validation_data = validation_generator, steps_per_epoch = 132,epochs = 100, validation_steps = 50, verbose = 1, callbacks=callbacks_list) #

   # return accuracy


Loaded model from disk
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
flatten_1 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               8388864   
_________________________________________________________________
dropout_4 (Dropout)          (N

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


132/132 [==============================] - 9s 61ms/step - loss: 0.9172 - acc: 0.4429 - val_loss: 0.7029 - val_acc: 0.5520
INFO:tensorflow:Assets written to: /gdrive/MyDrive/5644_ML_Project/Models_1/transfter_model_v2/assets
Epoch 2/100
132/132 [==============================] - 8s 60ms/step - loss: 0.8307 - acc: 0.4811 - val_loss: 0.6778 - val_acc: 0.6080
INFO:tensorflow:Assets written to: /gdrive/MyDrive/5644_ML_Project/Models_1/transfter_model_v2/assets
Epoch 3/100
132/132 [==============================] - 8s 63ms/step - loss: 0.7781 - acc: 0.5319 - val_loss: 0.6554 - val_acc: 0.6260
INFO:tensorflow:Assets written to: /gdrive/MyDrive/5644_ML_Project/Models_1/transfter_model_v2/assets
Epoch 4/100
132/132 [==============================] - 8s 62ms/step - loss: 0.7593 - acc: 0.5518 - val_loss: 0.6421 - val_acc: 0.6440
INFO:tensorflow:Assets written to: /gdrive/MyDrive/5644_ML_Project/Models_1/transfter_model_v2/assets
Epoch 5/100
132/132 [==============================] - 9s 65ms/step 

In [ ]:
loss , accuracy = model.evaluate(test_generator)

83/83 [==============================] - 221s 3s/step - loss: 0.6419 - acc: 0.5955
